<a href="https://colab.research.google.com/github/DeveshValluru/CSE_572/blob/main/HW3_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np


In [2]:
# implementing Euclidean distance

def Euclidean_distance(x1, x2):
  return np.sqrt(np.sum((x1-x2)**2))

In [3]:
# implementing Cosine distance

def cosine_distance(x1, x2):
    denom = (np.linalg.norm(x1) * np.linalg.norm(x2) + 1e-10)
    return 1 - np.dot(x1, x2) / denom

In [4]:
# implementing jaccard distance
def jaccard_distance(x1, x2):
    num = np.minimum(x1, x2).sum()
    den = np.maximum(x1, x2).sum() + 1e-10
    return 1 - num / den

In [5]:
def Kmeans(X, K,  dist_func,max_iters = 500, stop_type= "centroid"):
  n,d = X.shape
  centroids = X[np.random.choice(n, K, replace=False)]

  prev_centroids = np.zeros_like(centroids)
  SSE_list = []


  for iterations in range(max_iters):
    labels = []
    for x in X:
      dist = np.array([dist_func(x,c) for c in centroids])
      labels.append(np.argmin(dist))

    labels = np.array(labels)
    prev_centroids = centroids.copy()


    for k in range(K):
      points = X[labels == k]
      if len(points) == 0:
        centroids[k] = prev_centroids[k]
      else:
        centroids[k] = np.mean(points, axis=0)


    SSE = 0
    for i,x in enumerate(X):
      dist = dist_func(x, centroids[labels[i]])
      SSE += dist ** 2

    SSE_list.append(SSE)

    if stop_type == "centroid":
      if np.allclose(centroids, prev_centroids):
        break

    elif stop_type == "SSE":
      if iterations>0 and SSE_list[-1] > SSE_list[-2]:
        break

    elif stop_type == "max_iter":
      if iterations == max_iters - 1:
        break

  return centroids, labels, SSE_list





In [6]:
import pandas as pd

In [7]:
X_df = pd.read_csv("/content/data.csv", header=None)
X = X_df.values

X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
Y_df = pd.read_csv("/content/label.csv", header=None)
Y = Y_df.values

Y_df.head()

,0
0,7
1,2
2,1
3,0
4,4


In [9]:
K = len(np.unique(Y))
print("K =", K)


K = 10


In [10]:
cent_e, labels_e, SSE_e = Kmeans(X, K, Euclidean_distance)
cent_c, labels_c, SSE_c = Kmeans(X, K, cosine_distance)
cent_j, labels_j, SSE_j = Kmeans(X, K, jaccard_distance)

print(SSE_e[-1], SSE_c[-1], SSE_j[-1])


25320648645.0 682.1420945204158 3654.787351703579


Q1: Run K-means clustering with Euclidean, Cosine and Jarcard similarity. Specify K= the
number of categorical values of y (the number of classifications). Compare the SSEs of
Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method is better? (10 points)



In [11]:
print("Final Euclidean SSE:", SSE_e[-1])
print("Final Cosine SSE:", SSE_c[-1])
print("Final Jaccard SSE:", SSE_j[-1])


Final Euclidean SSE: 25320648645.0
Final Cosine SSE: 682.1420945204158
Final Jaccard SSE: 3654.787351703579


we have set k has the distinct classes in the labels. As you can see the SSE values for each distance type. Since the SSE value for the cosine Kmeans is the lowest it is the best performing in accordance with SSE criterion. Followed by Jaccard Kmeans. And finally Euclidean KMeans performs the worst among the three.

Q2: Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. First,
label each cluster using the majority vote label of the data points in that cluster. Later, compute
the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which metric
is better? (10 points)

In [12]:
def cluster_accuracy(labels, y, K):
    mapping = {}
    for k in range(K):
        cluster_points = y[labels == k]
        if len(cluster_points) == 0:
            mapping[k] = -1  # empty cluster
        else:
            mapping[k] = np.bincount(cluster_points).argmax()

    pred = np.array([mapping[l] for l in labels])
    acc = np.mean(pred == y)
    return acc


In [13]:
y = Y.ravel()

In [14]:
acc_e = cluster_accuracy(labels_e, y, K)
acc_c = cluster_accuracy(labels_c, y, K)
acc_j = cluster_accuracy(labels_j, y, K)

print("Euclidean Accuracy:", acc_e)
print("Cosine Accuracy:", acc_c)
print("Jaccard Accuracy:", acc_j)


Euclidean Accuracy: 0.5939
Cosine Accuracy: 0.6138
Jaccard Accuracy: 0.6025


All the accuracies are almost comparable as you can see the euclidean accuracy is 59.3%; cosine accuracy is 61.3% which is highest; Jaccard accuracy is 60.2%. Thus for this run cosine accuracy is highest for this random centroids picks. The run before this I got jaccard accuracy highest. So the accuracies depends on the dataset and also the random seed and the centorids we pick.

Q3: Set up the same stop criteria: “when there is no change in centroid position OR when the
SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you
can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-Kmeans, Jarcard-K-means. Which method requires more iterations and times to converge? (10
points)

In [15]:
iters_e = len(SSE_e)
iters_c = len(SSE_c)
iters_j = len(SSE_j)

print("Euclidean iterations:", iters_e)
print("Cosine iterations:", iters_c)
print("Jaccard iterations:", iters_j)


Euclidean iterations: 60
Cosine iterations: 86
Jaccard iterations: 66


Euclidean and Jaccard almost converges at the same time with 60 and 66 respectively. meaning Euclidean takes the least iterations to and is siglghtly faster than Jaccard. Both Jaccard and Euclidean outperforms and converges way faster than Cosine as it takes 86 iterations to cover and making it the slowest to converge.

Q4: Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to
the following three terminating conditions: (10 points)
• when there is no change in centroid position
• when the SSE value increases in the next iteration
• when the maximum preset value (e.g., 100) of iteration is complete

#no change in centroids

In [22]:
cent_e1, labels_e1, SSE_e1 = Kmeans(X, K, Euclidean_distance, stop_type="centroid")
cent_c1, labels_c1, SSE_c1 = Kmeans(X, K, cosine_distance, stop_type="centroid")
cent_j1, labels_j1, SSE_j1 = Kmeans(X, K, jaccard_distance, stop_type="centroid")


In [25]:
print("Final Euclidean SSE for no change in centroids:", SSE_e1[-1])
print("Final Cosine SSE for no change in centroids:", SSE_c1[-1])
print("Final Jaccard SSE for no change in centroids:", SSE_j1[-1])


Final Euclidean SSE for no change in centroids: 25410935787.0
Final Cosine SSE for no change in centroids: 685.3641042099662
Final Jaccard SSE for no change in centroids: 3666.3133171316244


# SSE Values increases for the next iteration



In [18]:
cent_e2, labels_e2, SSE_e2 = Kmeans(X, K, Euclidean_distance, stop_type="SSE")
cent_c2, labels_c2, SSE_c2 = Kmeans(X, K, cosine_distance, stop_type="SSE")
cent_j2, labels_j2, SSE_j2 = Kmeans(X, K, jaccard_distance, stop_type="SSE")


In [26]:
print("Final Euclidean SSE for SSE value increase:", SSE_e2[-1])
print("Final Cosine SSE for cSSE value increase:", SSE_c2[-1])
print("Final Jaccard SSE for SSE value increase:", SSE_j2[-1])


Final Euclidean SSE for SSE value increase: 25376215952.0
Final Cosine SSE for cSSE value increase: 688.9908584148054
Final Jaccard SSE for SSE value increase: 3686.5934403532437


#MAX preset iterations

In [30]:
cent_e3, labels_e3, SSE_e3 = Kmeans(X, K, Euclidean_distance, max_iters=100, stop_type="max_iter")
cent_c3, labels_c3, SSE_c3 = Kmeans(X, K, cosine_distance, max_iters=100, stop_type="max_iter")
cent_j3, labels_j3, SSE_j3 = Kmeans(X, K, jaccard_distance, max_iters=100, stop_type="max_iter")


In [31]:
print("Final Euclidean SSE for max preset iterations:", SSE_e3[-1])
print("Final Cosine SSE for max preset iterations:", SSE_c3[-1])
print("Final Jaccard SSE for max preset iterations:", SSE_j3[-1])


Final Euclidean SSE for max preset iterations: 25442203842.0
Final Cosine SSE for max preset iterations: 682.5187171822929
Final Jaccard SSE for max preset iterations: 3685.4249725089862


all cases are almost comparable with cosine SSE values being the best among the three and Euclidean SSE values being the worst. while jaccard being the next best after the Cosine distance SSE values.

Q5: What are your summary observations or takeaways based on your algorithmic analysis? (5
points)

Overall, the experiments show that the choice of distance metric has a clear impact on K-means behavior. Cosine K-means consistently achieved the lowest SSE, so it produced the most compact clusters in terms of within-cluster distance. All the accuracies are comparable to each other as they all have almost the same accuracies. We also observed that Cosine K-means required more iterations to converge than Euclidean and Jaccard, indicating a higher computational cost. Across all three stopping criteria, the SSE values for each metric were very similar, suggesting that most of the improvement happens well before the maximum iteration limit. Taken together, these results highlight a trade-off: Cosine distance is best for minimizing SSE, while  Cosine and Jaccard distance are almost comparable when we care about matching the true class labels. So if we want best of both worlds that is relative higher accuracy and also get them at faster pace we can choose Jaccard distance as it converges faster with 66 iterations, compared to cosine 86 iterations.